In [1]:
import pandas as pd
import numpy as np
import gc
import ml_metrics as metrics
#from sklearn import cross_validation
from sklearn import ensemble
from sklearn.decomposition import RandomizedPCA
#from sklearn.linear_model import LogisticRegression
#from sklearn.feature_selection import SelectKBest
#from sklearn.naive_bayes import GaussianNB
from sklearn import tree
#from sklearn.svm import SVC
#from sklearn.naive_bayes import BernoulliNB
#from datetime import datetime

In [ ]:
%%time
train_dtypes = {'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'is_booking': pd.np.int64,
'cnt': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64,
'hotel_cluster': pd.np.int64}

In [ ]:
%%time
all_train = pd.read_csv('train.csv', dtype=train_dtypes, iterator=True, chunksize=1000)
#iter_csv = pandas.read_csv('file.csv', iterator=True, chunksize=1000)
all_train = pd.concat([chunk[chunk['is_booking'] == 1] for chunk in all_train])
# all_train = pd.concat(all_train, ignore_index=True)
#temp_train = pd.read_csv('train.csv', dtype=train_dtypes, iterator=True, chunksize=1000)
#all_train = pd.concat(temp_train, ignore_index=True)

In [ ]:
%%time
all_train['id'] = [i for i in range(0, len(all_train))]
all_train['orig_destination_distance'] = all_train['orig_destination_distance'].fillna(-1)
all_train['date_time'] = pd.to_datetime(all_train['date_time'], errors='coerce')
all_train['srch_ci'] = pd.to_datetime(all_train['srch_ci'], errors='coerce')
all_train['srch_co'] = pd.to_datetime(all_train['srch_co'], errors='coerce')
all_train['activity_month'] = all_train['date_time'].fillna(-1).dt.month.astype(int)
all_train['activity_year'] = all_train['date_time'].fillna(-1).dt.year.astype(int)
all_train['checkin_month'] = all_train['srch_ci'].fillna(-1).dt.month.astype(int)
all_train['checkin_year'] = all_train['srch_ci'].fillna(-1).dt.year.astype(int)
all_train['checkout_month'] = all_train['srch_co'].fillna(-1).dt.month.astype(int)
all_train['checkout_year'] = all_train['srch_co'].fillna(-1).dt.year.astype(int)
#Split groups into two different classifiers for destinations vs. no destinations

In [2]:
%%time
destinations = pd.read_csv('destinations.csv')

CPU times: user 2.19 s, sys: 153 ms, total: 2.34 s
Wall time: 2.43 s


In [ ]:
%%time
destination_ids = destinations['srch_destination_id']
destination_ds = destinations.drop(['srch_destination_id'], 1)

In [ ]:
%%time
pca = RandomizedPCA(n_components=1, whiten=True).fit(destination_ds)

In [ ]:
%%time
destinations_pca = pca.transform(destination_ds)

In [ ]:
%%time
destinations_df = pd.DataFrame()
destinations_df['srch_destination_id'] = destination_ids
destinations_df['latent_destinations'] = destinations_pca

In [ ]:
%%time
with_dest_match = pd.merge(all_train, destinations_df)

In [ ]:
%%time
wo_dest_match = all_train[~(all_train.id.isin(with_dest_match.id))]

In [ ]:
%%time
with_features = with_dest_match.drop(['id', 'is_booking', 'cnt', 'user_id', 'hotel_cluster', 'date_time', 'srch_ci', 'srch_co'],1)
with_labels = with_dest_match['hotel_cluster']
wo_features = wo_dest_match.drop(['id', 'is_booking', 'cnt', 'user_id', 'hotel_cluster', 'date_time', 'srch_ci', 'srch_co'],1)
wo_labels = wo_dest_match['hotel_cluster']
with_features = with_features.reindex_axis(sorted(with_features.columns), axis=1)
wo_features = wo_features.reindex_axis(sorted(wo_features.columns), axis=1)

In [ ]:
# %%time
# pca_wo = RandomizedPCA(n_components=20, whiten=True).fit(wo_features)

In [ ]:
# %%time
# wo_train_features = pca_wo.transform(wo_features)

In [ ]:
%%time
#clf = LogisticRegression(tol=0.1)
#clf = GaussianNB()
#clf_with = ensemble.AdaBoostClassifier().fit(with_features, with_labels.values.ravel())
#clf_wo = ensemble.AdaBoostClassifier().fit(wo_features, wo_labels.values.ravel())
#clf = ensemble.AdaBoostClassifier(SVC(probability=True, kernel='linear'),n_estimators=10)
#clf = ensemble.GradientBoostingClassifier(SVC(probability=True, kernel='linear'),n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
#clf = ensemble.GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0, verbose=3)
# clf_with = tree.DecisionTreeClassifier(min_samples_split=100).fit(with_features, with_labels.values.ravel())
# clf_wo = tree.DecisionTreeClassifier(min_samples_split=50).fit(wo_features, wo_labels.values.ravel())
clf_with = ensemble.RandomForestClassifier(n_estimators=20, min_samples_split=500, n_jobs=2).fit(with_features, with_labels.values.ravel())
clf_wo = ensemble.RandomForestClassifier(n_estimators=20, min_samples_split=500, n_jobs=2).fit(wo_features, wo_labels.values.ravel())
#clf - BernoulliNB()

In [ ]:
# all_scores = []
# %%time
# for i in range(0, 20):
#     features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(features, labels, test_size=0.5)
#     clf = tree.DecisionTreeClassifier(min_samples_split=500)
#     clf = clf.fit(features_train, labels_train.values.ravel())
#     pred_probs = pd.DataFrame(clf.predict_proba(features_test))
#     pred_probs = pd.DataFrame([list([r.sort_values(ascending=False)[:3].index.values]) for i,r in pred_probs.iterrows()])
#     labels_test_df = pd.DataFrame(labels_test.values, index=range(0, len(labels_test)))
#     test_score = mapk(labels_test_df.values, pred_probs.values)
#     all_scores.append(test_score)
# print np.mean(all_scores)

In [ ]:
# %%time
#features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(features, labels, test_size=0.5)

In [ ]:
%%time
pred_probs = pd.DataFrame(clf.predict_proba(features_test))

In [ ]:
%%time
pred_probs = pd.DataFrame([list([r.sort_values(ascending=False)[:5].index.values]) for i,r in pred_probs.iterrows()])

In [ ]:
%%time
labels_test_df = pd.DataFrame(labels_test.values, index=range(0, len(labels_test)))

In [ ]:
%%time
test_score = mapk(labels_test_df.values, pred_probs.values)
print test_score

In [ ]:
%%time
test_dtypes = {'id': pd.np.int64,
'date_time': pd.np.object,
'site_name': pd.np.int64,
'posa_continent': pd.np.int64,
'user_location_country': pd.np.int64,
'user_location_region': pd.np.int64,
'user_location_city': pd.np.int64,
'orig_destination_distance': pd.np.float64,
'user_id': pd.np.int64,
'is_mobile': pd.np.int64,
'is_package': pd.np.int64,
'channel': pd.np.int64,
'srch_ci': pd.np.object,
'srch_co': pd.np.object,
'srch_adults_cnt': pd.np.int64,
'srch_children_cnt': pd.np.int64,
'srch_rm_cnt': pd.np.int64,
'srch_destination_id': pd.np.int64,
'srch_destination_type_id': pd.np.int64,
'hotel_continent': pd.np.int64,
'hotel_country': pd.np.int64,
'hotel_market': pd.np.int64}

In [ ]:
%%time
all_test = pd.read_csv('test.csv', dtype=test_dtypes)

In [ ]:
%%time
all_test['orig_destination_distance'] = all_test['orig_destination_distance'].fillna(-1)
all_test['date_time'] = pd.to_datetime(all_test['date_time'], errors='coerce')
all_test['srch_ci'] = pd.to_datetime(all_test['srch_ci'], errors='coerce')
all_test['srch_co'] = pd.to_datetime(all_test['srch_co'], errors='coerce')
all_test['activity_month'] = all_test['date_time'].fillna(-1).dt.month.astype(int)
all_test['checkin_month'] = all_test['srch_ci'].fillna(-1).dt.month.astype(int)
all_test['checkout_month'] = all_test['srch_co'].fillna(-1).dt.month.astype(int)

In [ ]:
%%time
with_dest_test = pd.merge(all_test, destinations_df)
with_testing_ids = with_dest_test['id'] 
with_testing_features = with_dest_test.drop(['user_id', 'id', 'date_time', 'srch_ci', 'srch_co'],1)

In [ ]:
%%time
wo_dest_test = all_test[~(all_test.id.isin(with_dest_test.id))]
wo_testing_ids = wo_dest_test['id']
wo_testing_features = wo_dest_test.drop(['user_id', 'id', 'date_time', 'srch_ci', 'srch_co'],1)

In [ ]:
%%time
with_testing_features = with_testing_features.reindex_axis(sorted(with_testing_features.columns), axis=1)
wo_testing_features = wo_testing_features.reindex_axis(sorted(wo_testing_features.columns), axis=1)

In [ ]:
# %%time
# with_test = pca_with.transform(with_testing_features)

In [ ]:
# %%time
# wo_test = pca_wo.transform(wo_testing_features)

In [ ]:
%%time
#feature_test_file = selector.transform(all_test)
#pred = clf.predict(feature_test_list)
with_test_probs = pd.DataFrame(clf_with.predict_proba(with_testing_features))
wo_test_probs = pd.DataFrame(clf_wo.predict_proba(wo_testing_features))

In [ ]:
%%time
with_test_probs = pd.Series([(i, r.sort_values(ascending=False)[:5].index.values) for i,r in with_test_probs.iterrows()])
with_test_probs = with_test_probs.values

In [ ]:
%%time
indices_1 = with_testing_ids.values
values_1 = [b for a,b in with_test_probs]

In [ ]:
%%time
submission_1 = pd.DataFrame()
submission_1['id'] = indices_1
submission_1['hotel_cluster'] = [' '.join(str(x) for x in y) for y in values_1]

In [ ]:
%%time
wo_test_probs = pd.Series([(i, r.sort_values(ascending=False)[:5].index.values) for i,r in wo_test_probs.iterrows()])
wo_test_probs = wo_test_probs.values

In [ ]:
%%time
indices_2 = wo_testing_ids.values
values_2 = [b for a,b in wo_test_probs]

In [ ]:
%%time
submission_2 = pd.DataFrame()
submission_2['id'] = indices_2
submission_2['hotel_cluster'] = [' '.join(str(x) for x in y) for y in values_2]

In [ ]:
%%time
submission = pd.concat([submission_1, submission_2])

In [ ]:
submission.sort_values(by='id', inplace=True)

In [ ]:
%%time
submission.to_csv('submission.csv', index=False)